In [23]:
#General packages useful in notebook 
import os
import pandas as pd
import warnings 
import numpy as np
import networkx as nx 

warnings.filterwarnings("ignore")

os.chdir('/workspaces/ROC-network-analysis/py-files/')


#This second part of validation_tool does not drop data but checks that the cleaned dataset has good data quality
from validation_tool import get_dataframe_into_ds,check_data_quality,assure_data_quality
#from validation_tool import get_peers_outside_class_warning

from datafy import get_isolated_child_inwards_per_class
from datafy import get_paired_ds
from datafy import get_reciprocal_friendship_ds
from datafy import get_reciprocity_total_nominations_frak
from datafy import get_isolated_outwards_info

os.chdir('/workspaces/ROC-network-analysis/')

In [24]:
# Not pretty solution to change name on follow_up_file.
FUR_0 = pd.read_csv("/workspaces/ROC-network-analysis/input-files/roc_network_data_follow_up.csv")

FUR_1 = FUR_0.rename(columns={'fs_school_id':'school_id',
                              'fs_school_no':'school_no', 
                              'fs_classroom_no':'classroom_no', 
                              'fs_classroom':'classroom_id',
                              'fs_student_id':'student_id',
                              'emot_1':'friend_1', 
                              'emot_2':'friend_2', 
                              'emot_3':'friend_3', 
                              'academic_1':'support_1',
                              'academic_2':'support_2', 
                              'academic_3':'support_3'})

FUR_1.to_csv("/workspaces/ROC-network-analysis/input-files/roc_network_data_follow_up2.csv",header=True, index=False)

In [ ]:
#Please change name
file_name = "roc_network_data_follow_up2.csv"
id_xlsx = False
## Full list the validation tool is run over
target_variables = ['school_id','classroom_id','student_id']


if file_name != 'follow_up.csv':
    final_df = pd.read_csv(f"/workspaces/ROC-network-analysis/input-files/{file_name}")

    final_df = assure_data_quality(get_dataframe_into_ds(#path = '/Users/network-analysis/input-files/',
                                                        path=os.getcwd()+"/input-files",
                                                        file_name= file_name, 
                                                        is_xlsx=id_xlsx),
                                                        target_variables=target_variables)

    final_df['group_id']=np.nan

    check_data_quality(final_df)

    #Keep all records, including those not in endline (this will be pass thru in stats for religious segregations but not in the others).
    final_df_with_no_el_also = final_df.copy()

    #Filter out endline (el) = no to avoid affecting statistics. 
    final_df_el_y = final_df[final_df['el']=='yes'].copy().reset_index(drop=True)
    final_df = final_df_el_y.copy()

else:
    df = pd.read_csv(f"/workspaces/ROC-network-analysis/input-files/{file_name}")
    final_df = df.dropna(axis = 0, how = 'all')

variable:  classroom_id , count_unique_values: 185
variable:  s_merge_id , count_unique_values: 11669
variable:  bs_m_score , count_unique_values: 16
variable:  school_id , count_unique_values: 127
variable:  student_id , count_unique_values: 6269
variable:  high_eyes , count_unique_values: 2
variable:  high_raven , count_unique_values: 2
variable:  bs_b_score , count_unique_values: 26
variable:  high_bangla , count_unique_values: 2
variable:  high_math , count_unique_values: 2
variable:  support_1 , count_unique_values: 3510
variable:  support_2 , count_unique_values: 3593
variable:  support_3 , count_unique_values: 3486
variable:  group_id , count_unique_values: 0


In [26]:
target_variables = ['friend_','support_']

for tv in target_variables:
    
    #output_name_with_destination = "/Users/network-analysis/output-files/Network_analysis"+"_"+str(tv)+".xlsx"
    output_name_with_destination = f"/workspaces/ROC-network-analysis/output-files/{file_name.split(".")[0]}_network_ouput"+"_"+str(tv)+".xlsx" 

    # Fraction of isolated children (inwards)
    isolated_inwards_ds, isolated_inwards_list_df = get_isolated_child_inwards_per_class(final_df,target_variable=tv)
    print("Isolation (inwards), per target variable " + str(tv))
    if isolated_inwards_ds.shape[0]>0:
        print("--status--OK")

    # Fraction of isolated children --this seems to work as it should, but spits out same over variables, so probably not right.
    isolated_frak_outwards,output_df_isolated_outwards = get_isolated_outwards_info(final_df,tv)
    print("Isolation (outwards), per target variable " + str(tv) )
    if isolated_frak_outwards.shape[0]>0:
        print("--status--OK")

    # Total reciprocity count and fraction (method 1) --this works as before on the development dataset, not on the new one tho.
    reciprocity_total_nominations = get_reciprocity_total_nominations_frak(get_paired_ds(final_df,target_variable=tv),
                                        get_reciprocal_friendship_ds(get_paired_ds(final_df,target_variable=tv)))



    print("Reciprocity total nominations, per target variable " + str(tv))
    if reciprocity_total_nominations.shape[0]>0:
        print("--status--OK")


    with pd.ExcelWriter(output_name_with_destination) as writer: 
            isolated_inwards_ds.to_excel(writer,sheet_name='Isolated_inwards_share',startrow=0, 
                                            startcol=0,index=False)
            
            reciprocity_total_nominations.to_excel(writer,sheet_name='Reciprocity_share',startrow=0, 
                                            startcol=0,index=False)

ZeroDivisionError: division by zero